# Market Room Type Classifier
In this notebook we experiment the NLP and K-means clustering functions for standarizing the room types from the OTA property scrapped data.

In [28]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

In [30]:
import os
import sys

sys.path.insert(1,"/home/ushan/workspace/rezaware/")
import rezaware as reza
from utils.modules.etl.load import sparkFILEwls as files
from utils.modules.ml.natlang import nlp 
from utils.modules.ml.cluster import points 


''' restart initiate classes '''
if debug:
    import importlib
    reza = importlib.reload(reza)
    rw = importlib.reload(files)
    nlp = importlib.reload(nlp)
    points = importlib.reload(points)
    
    
__desc__ = "cluster and classify room types"
clsRW = files.FileWorkLoads(desc=__desc__)
clsCL = points.ClusterWorkLoads(desc=__desc__)
clsNLP = nlp.NatLanWorkLoads(desc=__desc__)

#clsCL_1 = points.RoomType(desc=__desc__)

print("\nClass initialization and load complete!")

All functional APP-libraries in REZAWARE-package of REZAWARE-module imported successfully!
Some packages in ETL module LOAD package for FILEWORKLOADS function didn't load
No module named 'google.cloud'
All functional NATLANWORKLOADS-libraries in NATLANG-package of ML-module imported successfully!
All functional CLUSTERWORKLOADS-libraries in CLUSTER-package of ML-module imported successfully!
FileWorkLoads Class initialization complete
ClusterWorkLoads Class initialization complete
NatLanWorkLoads Class initialization complete

Class initialization and load complete!


## Read data into DataFrame

In [5]:
import os
import sys

sys.path.insert(1,"/home/ushan/workspace/rezaware/")
import rezaware as reza
from utils.modules.etl.load import sparkFILEwls as spark
from utils.modules.etl.load.sparkFILEwls import credentials as cred
import pandas as pd

clsSpark = spark.FileWorkLoads(desc=__desc__)

FileWorkLoads Class initialization complete


In [30]:
clsSpark.storeMode = "local-fs"
clsSpark.storeRoot = "/home/ushan/workspace/rezaware"

print("data_root =",clsSpark.storeRoot)


__as_type__ = "pandas"  # specify data type to return from read file

__local_folder_path__ = "wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0"

__local_file_name__ = None  #file name to read from or write data to

__file_type__ = "csv"   # specify file type to write data to or read from

''' options key value pairs must match the pyspark standard'''
options = {
    "inferSchema":True,
    "header":True,
    "delimiter":",",
}

_data = clsSpark.read_files_to_dtype(
    as_type=__as_type__,
    folder_path=__local_folder_path__,
    file_name=__local_file_name__,
    file_type=__file_type__,
    **options,
)


data_root = /home/ushan/workspace/rezaware
23/01/22 22:32:52 INFO InMemoryFileIndex: It took 17 ms to list leaf files for 1 paths.
23/01/22 22:32:52 INFO HadoopFSUtils: Listing leaf files and directories in parallel under 204 paths. The first several paths are: file:/home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20023488.2022-09-09.020.csv, file:/home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20088325.2022-09-09.030.csv, file:/home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20015732.2022-09-09.020.csv, file:/home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20050264.2022-09-09.030.csv, file:/home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20015732.2022-09-09.070.csv, file:/home/us

23/01/22 22:32:52 INFO TaskSetManager: Finished task 17.0 in stage 10.0 (TID 252) in 13 ms on ushan-Inspiron-5558.lan (executor driver) (18/204)
23/01/22 22:32:52 INFO Executor: Running task 18.0 in stage 10.0 (TID 253)
23/01/22 22:32:52 INFO Executor: Finished task 18.0 in stage 10.0 (TID 253). 1977 bytes result sent to driver
23/01/22 22:32:52 INFO TaskSetManager: Starting task 19.0 in stage 10.0 (TID 254) (ushan-Inspiron-5558.lan, executor driver, partition 19, PROCESS_LOCAL, 4595 bytes) taskResourceAssignments Map()
23/01/22 22:32:52 INFO Executor: Running task 19.0 in stage 10.0 (TID 254)
23/01/22 22:32:52 INFO TaskSetManager: Finished task 18.0 in stage 10.0 (TID 253) in 30 ms on ushan-Inspiron-5558.lan (executor driver) (19/204)
23/01/22 22:32:52 INFO Executor: Finished task 19.0 in stage 10.0 (TID 254). 1977 bytes result sent to driver
23/01/22 22:32:52 INFO TaskSetManager: Starting task 20.0 in stage 10.0 (TID 255) (ushan-Inspiron-5558.lan, executor driver, partition 20, PROCE

23/01/22 22:32:52 INFO Executor: Finished task 35.0 in stage 10.0 (TID 270). 1934 bytes result sent to driver
23/01/22 22:32:52 INFO TaskSetManager: Starting task 36.0 in stage 10.0 (TID 271) (ushan-Inspiron-5558.lan, executor driver, partition 36, PROCESS_LOCAL, 4595 bytes) taskResourceAssignments Map()
23/01/22 22:32:52 INFO Executor: Running task 36.0 in stage 10.0 (TID 271)
23/01/22 22:32:52 INFO TaskSetManager: Finished task 35.0 in stage 10.0 (TID 270) in 10 ms on ushan-Inspiron-5558.lan (executor driver) (36/204)
23/01/22 22:32:52 INFO Executor: Finished task 36.0 in stage 10.0 (TID 271). 1934 bytes result sent to driver
23/01/22 22:32:52 INFO TaskSetManager: Starting task 37.0 in stage 10.0 (TID 272) (ushan-Inspiron-5558.lan, executor driver, partition 37, PROCESS_LOCAL, 4595 bytes) taskResourceAssignments Map()
23/01/22 22:32:52 INFO TaskSetManager: Finished task 36.0 in stage 10.0 (TID 271) in 8 ms on ushan-Inspiron-5558.lan (executor driver) (37/204)
23/01/22 22:32:52 INFO E

23/01/22 22:32:53 INFO Executor: Finished task 57.0 in stage 10.0 (TID 292). 1934 bytes result sent to driver
23/01/22 22:32:53 INFO TaskSetManager: Starting task 58.0 in stage 10.0 (TID 293) (ushan-Inspiron-5558.lan, executor driver, partition 58, PROCESS_LOCAL, 4595 bytes) taskResourceAssignments Map()
23/01/22 22:32:53 INFO Executor: Running task 58.0 in stage 10.0 (TID 293)
23/01/22 22:32:53 INFO TaskSetManager: Finished task 57.0 in stage 10.0 (TID 292) in 9 ms on ushan-Inspiron-5558.lan (executor driver) (58/204)
23/01/22 22:32:53 INFO Executor: Finished task 58.0 in stage 10.0 (TID 293). 1977 bytes result sent to driver
23/01/22 22:32:53 INFO TaskSetManager: Starting task 59.0 in stage 10.0 (TID 294) (ushan-Inspiron-5558.lan, executor driver, partition 59, PROCESS_LOCAL, 4595 bytes) taskResourceAssignments Map()
23/01/22 22:32:53 INFO TaskSetManager: Finished task 58.0 in stage 10.0 (TID 293) in 24 ms on ushan-Inspiron-5558.lan (executor driver) (59/204)
23/01/22 22:32:53 INFO E


[Stage 10:=================>                                     (65 + 1) / 204]



...

23/01/22 22:32:57 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20090971.2022-09-09.010.csv, range: 0-7995, partition values: [empty row]
23/01/22 22:32:57 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20131185.2022-09-09.080.csv, range: 0-7975, partition values: [empty row]
23/01/22 22:32:57 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20131185.2022-09-09.090.csv, range: 0-7968, partition values: [empty row]
23/01/22 22:32:57 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20090971.2022-09-09.030.csv, range: 0-7967, partition values: [empty row]
23/01/22 22:

23/01/22 22:32:57 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20014181.2022-09-09.030.csv, range: 0-7609, partition values: [empty row]
23/01/22 22:32:57 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20014181.2022-09-09.040.csv, range: 0-7609, partition values: [empty row]
23/01/22 22:32:57 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20131185.2022-09-09.190.csv, range: 0-7596, partition values: [empty row]
23/01/22 22:32:57 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20131185.2022-09-09.180.csv, range: 0-7596, partition values: [empty row]
23/01/22 22:


[Stage 19:========================>                                 (3 + 1) / 7]



23/01/22 22:32:57 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20021296.2022-09-09.020.csv, range: 0-7102, partition values: [empty row]
23/01/22 22:32:57 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20014181.2022-09-09.160.csv, range: 0-7094, partition values: [empty row]
23/01/22 22:32:57 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20030916.2022-09-09.070.csv, range: 0-7093, partition values: [empty row]
23/01/22 22:32:57 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20021296.2022-09-09.070.csv, range: 0-7081, partition values: [empty row]
23/01/22 22:

23/01/22 22:32:58 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20088325.2022-09-09.060.csv, range: 0-6567, partition values: [empty row]
23/01/22 22:32:58 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20088325.2022-09-09.070.csv, range: 0-6567, partition values: [empty row]
23/01/22 22:32:58 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20088325.2022-09-09.050.csv, range: 0-6567, partition values: [empty row]
23/01/22 22:32:58 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20030916.2022-09-09.040.csv, range: 0-6553, partition values: [empty row]
23/01/22 22:

In [32]:
clsSpark.storeRoot

'/home/ushan/workspace/rezaware'

In [12]:
_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5087 entries, 0 to 5086
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   search_dt       5087 non-null   datetime64[ns]
 1   checkin_date    5087 non-null   datetime64[ns]
 2   property_name   5087 non-null   object        
 3   room_type       5087 non-null   object        
 4   room_rate       5087 non-null   object        
 5   review_score    5087 non-null   float64       
 6   destination_ID  5087 non-null   int32         
 7   location_desc   5087 non-null   object        
 8   other_info      5087 non-null   object        
dtypes: datetime64[ns](2), float64(1), int32(1), object(5)
memory usage: 337.9+ KB


In [13]:
_data.rename(columns = {'Search DateTime':'search_dt' , 'Checkin Date':'checkin_date', 'Property Name':'property_name' ,'Room Type':'room_type' ,'Room Rate':'room_rate' ,'Review Score':'review_score' ,'Destination ID':'destination_ID' ,'Location Desc':'location_desc' ,'Other Info': 'other_info'    }, inplace = True)



In [18]:
_data.head(1)

,search_dt,checkin_date,property_name,room_type,room_rate,review_score,destination_ID,location_desc,other_info
0,2022-09-09 16:14:05.613613,2022-09-09,Marriott's Grande Vista,King Room with Sofa Bed - Non-Smoking,US$169,8.7,20023488,"Sea World Orlando Area, OrlandoShow on map10.6...",King Room with Sofa Bed - Non-Smoking2 beds (1...


In [17]:
_data.tail(1)

,search_dt,checkin_date,property_name,room_type,room_rate,review_score,destination_ID,location_desc,other_info
5086,2022-09-09 16:14:05.613613,2022-09-09,THE TIKI SUITE at WAIKIKI BANYAN,One-Bedroom Apartment,US$225,9.0,20030916,"Waikiki, HonoluluShow on map3.3 miles from centre",One-Bedroom ApartmentEntire apartment • 1 bedr...


## Get Week No , Date Gap of records

In [19]:
    # def get_week_no(self, df, col_name):

    #     self.df= df.dropna()
    #     self.df[col_name] = pd.to_datetime(df[col_name]).dt.date
    #     self.df[col_name] = pd.to_datetime(df[col_name])
        
    #     return self.df 
    
    
    #################################################################################
    
#     def get_week_no(

#         df, 
#         col_name1, 
#         col_name2
       
#         ):
    
#         df= df.dropna()
#         df[col_name1] = pd.to_datetime(df[col_name1]).dt.date
#         df[col_name1] = pd.to_datetime(df[col_name1])
        
#         df[col_name2] = pd.to_datetime(df[col_name2]).dt.date
#         df[col_name2] = pd.to_datetime(df[col_name2])
        
#         df['Date_gap'] = df[col_name2] - df[col_name1]
#         # remove 'days' in 'Date_gap' column
#         df['Date_gap'] = df['Date_gap'].astype(str)
#         df["Date_gap"]= df["Date_gap"].replace( r"days","", regex=True)
        
#         return df 
    

In [20]:
# _data =  get_week_no(_data, 'search_dt', 'checkin_date')

In [31]:
_data =  clsCL.get_week_no( _data, 'search_dt', 'checkin_date')

## Get room type embeddings 

In [42]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('distilbert-base-nli-mean-tokens')

# _data=_data[_data.room_rate != 0]
# _data = _data.reset_index()
# corpus = list(_data['room_type'])

# corpus_embeddings = embedder.encode(corpus)



In [44]:
_se_props = {
    "NOSTOPWORDS":True,
    "LOWER":True,
}
_sent,_emb = clsNLP.get_sentence_embeddings(
    sentences=_data['property_name'],   # list of word ngrams
    model_name='distilbert-base-nli-mean-tokens',   # https://www.sbert.net/docs/pretrained_models.html
    kwargs=_se_props

)
print(_sent, "\n\n", _emb)

## Label data with kmeans clusters

In [161]:
__category__ = "KMEANS"
__columns__ = ['destination_id','review_score']
clust_props = {
    "NCLUSTERS":10,
    "MAXITERATIONS":200,
    "CENTROIDINIT":5,
    "RANDOMSTATE":0,
}
_cl_data = clsCL.cluster_n_label_data(
    data = _emb,#_data.dropna(),
    category = __category__,
    columns = __columns__,
    **clust_props
)
clsCL.clusters

<class 'numpy.ndarray'>


array([2, 9, 9, ..., 8, 9, 9], dtype=int32)

In [47]:
from sklearn.cluster import KMeans

In [48]:

num_clusters = 10
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_
cluster_assignment

array([1, 4, 7, ..., 7, 7, 0], dtype=int32)

In [51]:
cluster_df = pd.DataFrame(corpus, columns = ['corpus'])
cluster_df['cluster'] = cluster_assignment
cluster_df.head()

,corpus,cluster
0,King Room with Sofa Bed - Non-Smoking,1
1,King Suite-Hearing Access,4
2,Apartment,7
3,"Guest room, 1 King Bed",0
4,King Room,4


## Combine dataframes with updated room types

In [55]:
# Merge two Dataframes on index of both the dataframes
_data= _data.merge(cluster_df, left_index=True, right_index=True)

In [57]:
_data=_data.drop([ 'index', ], axis=1)

In [58]:
_data['new_room_type'] = ''
room_list=['King suite studio Smoking- city view',
'King suite standard studio- 1 Double Delux bed - No smoking',
'Queen suite - 2 standard beds-Guest Sofa',
'Queen superior suite studio with 2 beds',
'Queen suit Delux -2 Double beds - Non Smoking',
'King standard superior',
'Delux King suite Double bed-city View',
'Double Standard delux room 2 Beds - city view',
'King - Access Disability ',
'King suite with Sofa Dormitory - 1 Bed']


for i in range(10):    
    _data['new_room_type'][_data["cluster"]==i]=room_list[i]

In [60]:
_data.head()

,search_dt,checkin_date,property_name,room_type,room_rate,review_score,destination_ID,location_desc,other_info,Date_gap,corpus,cluster,new_room_type
0,2022-09-09,2022-09-09,Marriott's Grande Vista,King Room with Sofa Bed - Non-Smoking,US$169,8.7,20023488,"Sea World Orlando Area, OrlandoShow on map10.6...",King Room with Sofa Bed - Non-Smoking2 beds (1...,0,King Room with Sofa Bed - Non-Smoking,1,King suite standard studio- 1 Double Delux bed...
1,2022-09-09,2022-09-09,Embassy Suites Orlando - North,King Suite-Hearing Access,US$169,7.9,20023488,OrlandoShow on map8.9 miles from centre,King Suite-Hearing AccessPrivate suite2 beds (...,0,King Suite-Hearing Access,4,Queen suit Delux -2 Double beds - Non Smoking
2,2022-09-09,2022-09-09,Cozy 1BR with Two Queen Beds - Pool and Hot Tu...,Apartment,US$77,5.0,20023488,"International Drive, OrlandoShow on map7.7 mil...",ApartmentEntire apartment • 1 bedroom • 1 livi...,0,Apartment,7,Double Standard delux room 2 Beds - city view
3,2022-09-09,2022-09-09,"Grand Bohemian Hotel Orlando, Autograph Collec...","Guest room, 1 King Bed",US$229,8.5,20023488,"Downtown Orlando, OrlandoShow on map0.4 miles ...","Guest room, 1 King Bed1 extra-large double bed...",0,"Guest room, 1 King Bed",0,King suite studio Smoking- city view
4,2022-09-09,2022-09-09,Quality Inn & Suites Near the Theme Parks,King Room,US$149,7.7,20023488,"International Drive, OrlandoShow on map5.6 mil...",King Room1 extra-large double bedBreakfast inc...,0,King Room,4,Queen suit Delux -2 Double beds - Non Smoking
